# Build chatbot

## Introduction

### Objective
Use GPT-3.5, Langchain and ChromaDB (&SQLite) to create Retrieval Augmented Generation (RAG) system.
In particular, it was implemented using KT AIVLE School's FAQ data to allow 6th class applicants to easily check information about KT AIVLE school.

### Development details
* **OS**: Windows 11 Pro
* **RAG Library**: ConversationalRetrievalChain
* **QA model**: gpt-3.5-turbo
* **Embedding model**: text-embedding-ada-002
* **DB**: SQLite3


## Environment

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Check OpenAI API Key
My API key has been masked, and you must save your API Key as 'OPENAI_API_KEY' in the Windows environment variable and then load it.

In [2]:
api_key = os.getenv('OPENAI_API_KEY')
print(api_key[:3]) 

sk-


Or uncomment the code below and insert your API key.

In [ ]:
# os.environ['OPENAI_API_KEY'] = 'your_API_key' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## Retrieval Augmetned Generation

### 1. Create VectorDB
This VectorDB will be used as a retriever later. The csv file was created through crawling and preprocessing, and the code can be checked through the 'crawl_faq.ipynb' file.

In [3]:
# Load data
data1 = pd.read_csv('faq_for_aivler.csv', encoding='utf-8', index_col=0)
data2 = pd.read_csv('faq_for_candidate.csv', encoding='utf-8', index_col=0)

# Merge 
total_df = pd.concat([data1, data2])

print('Total length:', len(total_df))
total_df.sample(3)

Total length: 82


,구분,QA
30,교육/수강,비대면으로 진행된다면 개인이 원하는 시간대에 참여할 수 있나요?\nKT 에이블스쿨은...
63,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\n네. AICE (기존 AI...
26,교육/수강,교육 수료 기준은 무엇인가요?\n<p>KT 에이블스쿨은 고용노동부 K-Digital...


In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./db",        # 경로 지정(현 위치에서 db 폴더 생성)
                  embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

In [5]:
doc = [Document(page_content = key+'에 관한 내용입니다.\n'+value) for key, value in total_df.values]
idx = database.add_documents(doc)

Extract the k results most similar to the query from the database through cosine similarity.

In [6]:
k = 3
query = "저는 군인인데 신청 가능한가요?"

result = database.similarity_search_with_score(query, k = k)
print(result)
print('-'*50)
for doc in result:
    print(f"- Similarity : {round(doc[1], 5)}\n- Doc: {doc[0].page_content}") # 문서 내용 표시
    print()

[(Document(page_content='모집/선발에 관한 내용입니다.\n현재 군 복무중인 상태인데, 지원이 가능한가요?\n현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.'), 0.2954519987106323), (Document(page_content='모집/선발에 관한 내용입니다.\n현재 군 복무중인 상태인데, 지원이 가능한가요?\n현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.'), 0.295506645278315), (Document(page_content='모집/선발에 관한 내용입니다.\n이전에 불합격했을 경우 이번에 다시 지원할수 있나요?\n지원자격에 해당되는 경우 다시 지원이 가능합니다.'), 0.3525104820728302)]
--------------------------------------------------
- Similarity : 0.29545
- Doc: 모집/선발에 관한 내용입니다.
현재 군 복무중인 상태인데, 지원이 가능한가요?
현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.

- Similarity : 0.29551
- Doc: 모집/선발에 관한 내용입니다.
현재 군 복무중인 상태인데, 지원이 가능한가요?
현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.

- Similarity : 0.35251
- Doc: 모집/선발에 관한 내용입니다.
이전에 불합격했을 경우 이번에 다시 지원할수 있나요?
지원자격에 해당되는 경우 다시 지원이 가능합니다.



### 2. RAG+LLM

In [7]:
# chat_log: Create DB to store chat history
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS chat_log (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL,
    sim2 REAL,
    sim3 REAL,
    answer INTEGER NOT NULL,
    session_id INTEGER NOT NULL
)
''')

conn.commit()
conn.close()

In [8]:
# Retrieval
retriever = database.as_retriever(search_kwargs = {"k": k})

# Conversation memory for model
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

# QA Model: gpt-3.5-turbo
chat = ChatOpenAI(model="gpt-3.5-turbo")
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")



### 3. Use model

In [9]:
# Connect DB
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

while True:
    query = input('Query > ')
    query = query.strip()
    
    # dt: Time when query was entered
    dt = datetime.now()
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # PRINT ::: QUERY
    print(f'Query : {query}')
    
    # Get model results
    if len(query) == 0:
        break
    qa_result = qa({"question": query}) # 모델 대답
    retriever_result = database.similarity_search_with_score(query, k = 3) # Retriever가 찾은 3개의 답변과 점수
    scores = [round(doc[1], 5) for doc in result]
    
    # Save results in DB
    data = pd.DataFrame({'datetime': [dt], 'query': [query], 'answer': [qa_result['answer']],
                         'sim1': [scores[0]], 'sim2': [scores[1]], 'sim3': [scores[2]], 'session_id': [-1]})
    
    data.to_sql('chat_log', conn, if_exists='append', index=False) 
    
    # PRINT ::: ANSWER
    print(f'답변 : {qa_result["answer"]}')
    print('=' * 100)


# Commit changes & close connection to DB
conn.commit()
conn.close()

# # Clear memory of QA model
# memory.clear()

Query : 안녕하세요?
답변 : 안녕하세요! 무엇을 도와드릴까요?
Query : 저는 현재 군인이에요, 지원이 가능한가요?
답변 : 현재 군 복무중인 상태여도 지원 가능합니다. 다만, 교육 시작 전일까지 병역의무를 마쳐야 교육 수강이 가능합니다.
Query : 에이블스쿨에 지원할 때 별도의 시험이나 면접 전형이 존재하나요?
답변 : KT 에이블스쿨은 별도의 면접 과정이 없습니다.
Query : 시험은 있는 건가요?
답변 : KT 에이블스쿨의 교육과정 중에는 시험이 따로 있지는 않습니다. 교과목 종료 시에는 셀프테스트를 통한 역량 점검이 진행됩니다.
Query : 지원할 때 시험이 있는지 궁금해요
답변 : 죄송합니다, 해당 질문에 대한 답변을 드릴 수 없습니다.
Query : 알겠습니다, 감사합니다.
답변 : KT 에이블스쿨은 미취업자를 대상으로 하는 프로그램이며, 재직자는 지원이 불가능하다고 합니다. 그러나 시험과 관련된 구체적인 내용은 제공되지 않았습니다. 따라서, 시험 여부에 대해서는 정확한 정보를 얻기 위해 KT 에이블스쿨의 공식 웹사이트나 담당 부서에 문의하는 것이 좋을 것 같습니다.
Query : 
